In [1]:
import pandas as pd
import numpy as np
import datetime
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import dash
import dash_core_components as dcc
import dash_html_components as html

import ibmdata

In [50]:
daysback = 365
starttime = time.time()
Query = f"""
WITH cte AS(
    SELECT lot_Id, wafer_Id, tw.Calcdefs AS Sail, tw.Last_test_date AS date,
        unitcell_X - 1 AS retx, unitcell_Y AS rety,
        --(unitcell_X - 1)||'_'||unitcell_Y AS retXY,
        CASE WHEN LENGTH(parm_Label) = 37 THEN tw.testProgramName||'_0p75'||RIGHT(parm_Label,11) ELSE tw.testProgramName||'_0p75'||RIGHT(parm_Label,12) END AS label,
        parmYield as yield
    FROM DMIW.ChipParmFactR cpfr
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
        JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
    WHERE Last_test_date >= (current date - {daysback} days)
        AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL11', 'SSL21', 'SSL31')
        AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
        AND (UCASE(SUBSTR(tp.parm_Label,10,17)) = 'SSCANLATCH0P75VFM')
        AND ((unitcell_X - 1) || '_' || unitcell_Y NOT IN ('-2_-1','1_8','2_9','2_10','3_10','4_11','5_11','6_11','7_11','9_10','10_9','11_8','1_4','2_3','3_2','4_1','5_1','7_1','9_2','10_3','11_4'))
        AND parmYield < 100
        --AND wafer_id = 'AZ6B5-19'
    ORDER BY lot_Id, wafer_Id, parm_Label
),
lot_name_cte AS(
    SELECT DISTINCT wafer_id, E.CELL_NAME AS Lot_Name
    FROM DMIW_SYSTEMS.WAFER A,
        DMIW_SYSTEMS.PRODUCT B,
        DMIW_SYSTEMS.EWR_EVENT_FACT C,
        DMIW_SYSTEMS.EWR_EVENT D,
        DMIW_SYSTEMS.EWR_CELL E
    WHERE A.WAFERKEY=C.WAFERKEY
        AND B.PRODUCTKEY=C.PRODUCTKEY
        AND  C.EWR_CELLKEY=E.EWR_CELLKEY
        AND OWNER_USER_ID='hai.zhu@ibm.com'
        AND B.FAMILY_CODE = 'Q5'
        AND EWR_NAME IN ('7HPP_Name_P10')
        AND CELL_ID LIKE '7HPP_Name_P10%'
        AND E.CELL_NAME NOT IN ('Lot0-1','Lot0-2','uncertain')
)
SELECT lot_id, cte.wafer_id, MIN(date) AS sail_date, retx, rety,
--SELECT lot_name, lot_id, cte.wafer_id, retx, rety,
    SUM(
        CASE 
            WHEN Sail = 'SSL11' AND RIGHT(label,10) IN ('block6_so0','block6_so5','block7_so0','block7_so5') THEN 1
            WHEN Sail = 'SSL11' AND SUBSTR(label,11,7) IN ('block22','block24','block25') THEN 1
            WHEN Sail = 'SSL11' AND (RIGHT(label,10) NOT IN ('block6_so0','block6_so5','block7_so0','block7_so5') OR SUBSTR(label,11,7) NOT IN ('block22','block24','block25')) THEN -0.3
            ELSE 0 END
        ) AS SSL1_GF_metric,
    SUM(
        CASE 
            WHEN Sail = 'SSL21' AND RIGHT(label,10) IN ('block6_so0','block6_so5','block7_so0','block7_so5') THEN 1
            WHEN Sail = 'SSL21' AND SUBSTR(label,11,7) IN ('block22','block24','block25') THEN 1
            WHEN Sail = 'SSL21' AND (RIGHT(label,10) NOT IN ('block6_so0','block6_so5','block7_so0','block7_so5') OR SUBSTR(label,11,7) NOT IN ('block22','block24','block25')) THEN -0.3
            ELSE 0 END
        ) AS SSL2_GF_metric,
    SUM(
        CASE 
            WHEN Sail = 'SSL31' AND RIGHT(label,10) IN ('block6_so0','block6_so5','block7_so0','block7_so5') THEN 1
            WHEN Sail = 'SSL31' AND SUBSTR(label,11,7) IN ('block22','block24','block25') THEN 1
            WHEN Sail = 'SSL31' AND (RIGHT(label,10) NOT IN ('block6_so0','block6_so5','block7_so0','block7_so5') OR SUBSTR(label,11,7) NOT IN ('block22','block24','block25')) THEN -0.3
            ELSE 0 END
        ) AS SSL3_GF_metric
FROM cte
--JOIN lot_name_cte ln ON ln.wafer_id = cte.wafer_id
--GROUP BY lot_name, lot_id, cte.wafer_id, retx, rety
GROUP BY lot_id, cte.wafer_id, retx, rety
ORDER BY MIN(date) DESC, lot_id, cte.wafer_id
"""
df = ibmdata.isdw.query(Query)
df['ssl1_gf_metric'] = df['ssl1_gf_metric'].astype(float).round(2)
df['ssl2_gf_metric'] = df['ssl2_gf_metric'].astype(float).round(2)
df['ssl3_gf_metric'] = df['ssl3_gf_metric'].astype(float).round(2)
df['GF_metric'] = df['ssl1_gf_metric']+df['ssl2_gf_metric']+df['ssl3_gf_metric']
df['GF_metric'] = df['GF_metric'].round(2)
df['Ghost_Fin'] = df['GF_metric'].apply(lambda x: 1 if x > 1 else 0)
print(time.time()-starttime)

7.181213140487671


In [3]:
# Wafer Level Ghost Fin data
#agg_df = df.groupby(['lot_name','lot_id','wafer_id'])['Ghost_Fin'].sum().reset_index()
agg_df = df.groupby(['lot_id','wafer_id','sail_date'])['Ghost_Fin'].sum().reset_index()
agg_df['Ghost_Fin_YieldLoss'] = agg_df['Ghost_Fin'].apply(lambda x: 100*x/75).round(2)
agg_df = agg_df.rename(columns={"Ghost_Fin": "Ghost_Fin_ChipCount"})
agg_df = agg_df.sort_values('sail_date',ascending=False)

In [4]:
agg_df

,lot_id,wafer_id,sail_date,Ghost_Fin_ChipCount,Ghost_Fin_YieldLoss
545,AZ75RB.1,AZ75R-25,2020-10-23,1,1.33
540,AZ75RB.1,AZ75R-18,2020-10-23,4,5.33
529,AZ75RB.1,AZ75R-01,2020-10-23,1,1.33
530,AZ75RB.1,AZ75R-02,2020-10-23,3,4.00
531,AZ75RB.1,AZ75R-03,2020-10-23,1,1.33
...,...,...,...,...,...
29,AZ6PK.1,AZ6PK-10,2020-06-27,0,0.00
15,AZ6NN.1,AZ6NN-15,2020-06-25,9,12.00
14,AZ6NN.1,AZ6NN-03,2020-06-25,8,10.67
0,AZ6B5.1,AZ6B5-19,2020-06-24,18,24.00


In [11]:
gf_wafers = list(agg_df[agg_df.Ghost_Fin_ChipCount > 2].wafer_id)

In [45]:
daysback = 365
starttime = time.time()
Query = f"""
WITH cte AS(
    SELECT lot_Id, wafer_Id, unitcell_X - 1 AS retx, unitcell_Y AS rety,
        tw.Calcdefs AS Sail,
        tw.testProgramName||'_1p05'||RIGHT(parm_Label,10) AS label,
        parmYield as yield
    FROM DMIW.ChipParmFactR cpfr
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
        JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
    WHERE Last_test_date >= (current date - {daysback} days)
        AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL11', 'SSL21', 'SSL31')
        AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
        AND (UCASE(SUBSTR(tp.parm_Label,28,5)) IN ('SEL31','SEL32','SEL33','SEL34','SEL35','SEL36','SEL37','SEL38','SEL39','SEL40','SEL41','SEL42'))
        AND ((unitcell_X - 1) || '_' || unitcell_Y NOT IN ('-2_-1','1_8','2_9','2_10','3_10','4_11','5_11','6_11','7_11','9_10','10_9','11_8','1_4','2_3','3_2','4_1','5_1','7_1','9_2','10_3','11_4'))
        --AND parmYield < 100
    ORDER BY lot_Id, wafer_Id, parm_Label
)
SELECT *
FROM cte
"""
pack_df = ibmdata.isdw.query(Query)
print(time.time()-starttime)

In [57]:
pack_df

,lot_id,wafer_id,retx,rety,sail,label,yield
0,AZ6B5.1,AZ6B5-19,3,7,SSL11,SSL1_1p05_sel31_so0,100.00
1,AZ6B5.1,AZ6B5-19,3,9,SSL11,SSL1_1p05_sel31_so0,100.00
2,AZ6B5.1,AZ6B5-19,4,5,SSL11,SSL1_1p05_sel31_so0,100.00
3,AZ6B5.1,AZ6B5-19,8,9,SSL11,SSL1_1p05_sel31_so0,100.00
4,AZ6B5.1,AZ6B5-19,4,3,SSL11,SSL1_1p05_sel31_so0,100.00
...,...,...,...,...,...,...,...
9337099,AZ75RB.1,AZ75R-25,6,10,SSL31,SSL3_1p05_sel42_so5,100.00
9337100,AZ75RB.1,AZ75R-25,10,6,SSL31,SSL3_1p05_sel42_so5,100.00
9337101,AZ75RB.1,AZ75R-25,7,2,SSL31,SSL3_1p05_sel42_so5,100.00
9337102,AZ75RB.1,AZ75R-25,11,6,SSL31,SSL3_1p05_sel42_so5,100.00


In [58]:
table = pd.pivot_table(pack_df, values='yield', index=['lot_id','wafer_id','retx','rety'],columns=['label'], aggfunc=np.sum).reset_index()

In [59]:
table

label,lot_id,wafer_id,retx,rety,SSL1_1p05_sel31_so0,SSL1_1p05_sel31_so1,SSL1_1p05_sel31_so2,SSL1_1p05_sel31_so3,SSL1_1p05_sel31_so4,SSL1_1p05_sel31_so5,SSL1_1p05_sel32_so0,SSL1_1p05_sel32_so1,SSL1_1p05_sel32_so2,SSL1_1p05_sel32_so3,SSL1_1p05_sel32_so4,SSL1_1p05_sel32_so5,SSL1_1p05_sel33_so0,SSL1_1p05_sel33_so1,SSL1_1p05_sel33_so2,SSL1_1p05_sel33_so3,SSL1_1p05_sel33_so4,SSL1_1p05_sel33_so5,SSL1_1p05_sel34_so0,SSL1_1p05_sel34_so1,SSL1_1p05_sel34_so2,SSL1_1p05_sel34_so3,SSL1_1p05_sel34_so4,SSL1_1p05_sel34_so5,SSL1_1p05_sel35_so0,SSL1_1p05_sel35_so1,SSL1_1p05_sel35_so2,SSL1_1p05_sel35_so3,SSL1_1p05_sel35_so4,SSL1_1p05_sel35_so5,SSL1_1p05_sel36_so0,SSL1_1p05_sel36_so1,SSL1_1p05_sel36_so2,SSL1_1p05_sel36_so3,SSL1_1p05_sel36_so4,SSL1_1p05_sel36_so5,SSL1_1p05_sel37_so0,SSL1_1p05_sel37_so1,SSL1_1p05_sel37_so2,SSL1_1p05_sel37_so3,SSL1_1p05_sel37_so4,SSL1_1p05_sel37_so5,SSL1_1p05_sel38_so0,SSL1_1p05_sel38_so1,SSL1_1p05_sel38_so2,SSL1_1p05_sel38_so3,SSL1_1p05_sel38_so4,SSL1_1p05_sel38_so5,SSL1_1p05_sel39_so0,SSL1_1p05_sel39_so1,SSL1_1p05_sel39_so2,SSL1_1p05_sel39_so3,SSL1_1p05_sel39_so4,SSL1_1p05_sel39_so5,SSL1_1p05_sel40_so0,SSL1_1p05_sel40_so1,SSL1_1p05_sel40_so2,SSL1_1p05_sel40_so3,SSL1_1p05_sel40_so4,SSL1_1p05_sel40_so5,SSL1_1p05_sel41_so0,SSL1_1p05_sel41_so1,SSL1_1p05_sel41_so2,SSL1_1p05_sel41_so3,SSL1_1p05_sel41_so4,SSL1_1p05_sel41_so5,SSL1_1p05_sel42_so0,SSL1_1p05_sel42_so1,SSL1_1p05_sel42_so2,SSL1_1p05_sel42_so3,SSL1_1p05_sel42_so4,SSL1_1p05_sel42_so5,SSL2_1p05_sel31_so0,SSL2_1p05_sel31_so1,SSL2_1p05_sel31_so2,SSL2_1p05_sel31_so3,SSL2_1p05_sel31_so4,SSL2_1p05_sel31_so5,SSL2_1p05_sel32_so0,SSL2_1p05_sel32_so1,SSL2_1p05_sel32_so2,SSL2_1p05_sel32_so3,SSL2_1p05_sel32_so4,SSL2_1p05_sel32_so5,SSL2_1p05_sel33_so0,SSL2_1p05_sel33_so1,SSL2_1p05_sel33_so2,SSL2_1p05_sel33_so3,SSL2_1p05_sel33_so4,SSL2_1p05_sel33_so5,SSL2_1p05_sel34_so0,SSL2_1p05_sel34_so1,SSL2_1p05_sel34_so2,SSL2_1p05_sel34_so3,SSL2_1p05_sel34_so4,SSL2_1p05_sel34_so5,SSL2_1p05_sel35_so0,SSL2_1p05_sel35_so1,SSL2_1p05_sel35_so2,SSL2_1p05_sel35_so3,SSL2_1p05_sel35_so4,SSL2_1p05_sel35_so5,SSL2_1p05_sel36_so0,SSL2_1p05_sel36_so1,SSL2_1p05_sel36_so2,SSL2_1p05_sel36_so3,SSL2_1p05_sel36_so4,SSL2_1p05_sel36_so5,SSL2_1p05_sel37_so0,SSL2_1p05_sel37_so1,SSL2_1p05_sel37_so2,SSL2_1p05_sel37_so3,SSL2_1p05_sel37_so4,SSL2_1p05_sel37_so5,SSL2_1p05_sel38_so0,SSL2_1p05_sel38_so1,SSL2_1p05_sel38_so2,SSL2_1p05_sel38_so3,SSL2_1p05_sel38_so4,SSL2_1p05_sel38_so5,SSL2_1p05_sel39_so0,SSL2_1p05_sel39_so1,SSL2_1p05_sel39_so2,SSL2_1p05_sel39_so3,SSL2_1p05_sel39_so4,SSL2_1p05_sel39_so5,SSL2_1p05_sel40_so0,SSL2_1p05_sel40_so1,SSL2_1p05_sel40_so2,SSL2_1p05_sel40_so3,SSL2_1p05_sel40_so4,SSL2_1p05_sel40_so5,SSL2_1p05_sel41_so0,SSL2_1p05_sel41_so1,SSL2_1p05_sel41_so2,SSL2_1p05_sel41_so3,SSL2_1p05_sel41_so4,SSL2_1p05_sel41_so5,SSL2_1p05_sel42_so0,SSL2_1p05_sel42_so1,SSL2_1p05_sel42_so2,SSL2_1p05_sel42_so3,SSL2_1p05_sel42_so4,SSL2_1p05_sel42_so5,SSL3_1p05_sel31_so0,SSL3_1p05_sel31_so1,SSL3_1p05_sel31_so2,SSL3_1p05_sel31_so3,SSL3_1p05_sel31_so4,SSL3_1p05_sel31_so5,SSL3_1p05_sel32_so0,SSL3_1p05_sel32_so1,SSL3_1p05_sel32_so2,SSL3_1p05_sel32_so3,SSL3_1p05_sel32_so4,SSL3_1p05_sel32_so5,SSL3_1p05_sel33_so0,SSL3_1p05_sel33_so1,SSL3_1p05_sel33_so2,SSL3_1p05_sel33_so3,SSL3_1p05_sel33_so4,SSL3_1p05_sel33_so5,SSL3_1p05_sel34_so0,SSL3_1p05_sel34_so1,SSL3_1p05_sel34_so2,SSL3_1p05_sel34_so3,SSL3_1p05_sel34_so4,SSL3_1p05_sel34_so5,SSL3_1p05_sel35_so0,SSL3_1p05_sel35_so1,SSL3_1p05_sel35_so2,SSL3_1p05_sel35_so3,SSL3_1p05_sel35_so4,SSL3_1p05_sel35_so5,SSL3_1p05_sel36_so0,SSL3_1p05_sel36_so1,SSL3_1p05_sel36_so2,SSL3_1p05_sel36_so3,SSL3_1p05_sel36_so4,SSL3_1p05_sel36_so5,SSL3_1p05_sel37_so0,SSL3_1p05_sel37_so1,SSL3_1p05_sel37_so2,SSL3_1p05_sel37_so3,SSL3_1p05_sel37_so4,SSL3_1p05_sel37_so5,SSL3_1p05_sel38_so0,SSL3_1p05_sel38_so1,SSL3_1p05_sel38_so2,SSL3_1p05_sel38_so3,SSL3_1p05_sel38_so4,SSL3_1p05_sel38_so5,SSL3_1p05_sel39_so0,SSL3_1p05_sel39_so1,SSL3_1p05_sel39_so2,SSL3_1p05_sel39_so3,SSL3_1p05_sel39_so4,SSL3_1p05_sel39_so5,SSL3_1p0

In [60]:
df[df.Ghost_Fin == 1].merge(table, how='left', on=['lot_id','wafer_id','retx','rety'])

,lot_id,wafer_id,sail_date,retx,rety,ssl1_gf_metric,ssl2_gf_metric,ssl3_gf_metric,GF_metric,Ghost_Fin,SSL1_1p05_sel31_so0,SSL1_1p05_sel31_so1,SSL1_1p05_sel31_so2,SSL1_1p05_sel31_so3,SSL1_1p05_sel31_so4,SSL1_1p05_sel31_so5,SSL1_1p05_sel32_so0,SSL1_1p05_sel32_so1,SSL1_1p05_sel32_so2,SSL1_1p05_sel32_so3,SSL1_1p05_sel32_so4,SSL1_1p05_sel32_so5,SSL1_1p05_sel33_so0,SSL1_1p05_sel33_so1,SSL1_1p05_sel33_so2,SSL1_1p05_sel33_so3,SSL1_1p05_sel33_so4,SSL1_1p05_sel33_so5,SSL1_1p05_sel34_so0,SSL1_1p05_sel34_so1,SSL1_1p05_sel34_so2,SSL1_1p05_sel34_so3,SSL1_1p05_sel34_so4,SSL1_1p05_sel34_so5,SSL1_1p05_sel35_so0,SSL1_1p05_sel35_so1,SSL1_1p05_sel35_so2,SSL1_1p05_sel35_so3,SSL1_1p05_sel35_so4,SSL1_1p05_sel35_so5,SSL1_1p05_sel36_so0,SSL1_1p05_sel36_so1,SSL1_1p05_sel36_so2,SSL1_1p05_sel36_so3,SSL1_1p05_sel36_so4,SSL1_1p05_sel36_so5,SSL1_1p05_sel37_so0,SSL1_1p05_sel37_so1,SSL1_1p05_sel37_so2,SSL1_1p05_sel37_so3,SSL1_1p05_sel37_so4,SSL1_1p05_sel37_so5,SSL1_1p05_sel38_so0,SSL1_1p05_sel38_so1,SSL1_1p05_sel38_so2,SSL1_1p05_sel38_so3,SSL1_1p05_sel38_so4,SSL1_1p05_sel38_so5,SSL1_1p05_sel39_so0,SSL1_1p05_sel39_so1,SSL1_1p05_sel39_so2,SSL1_1p05_sel39_so3,SSL1_1p05_sel39_so4,SSL1_1p05_sel39_so5,SSL1_1p05_sel40_so0,SSL1_1p05_sel40_so1,SSL1_1p05_sel40_so2,SSL1_1p05_sel40_so3,SSL1_1p05_sel40_so4,SSL1_1p05_sel40_so5,SSL1_1p05_sel41_so0,SSL1_1p05_sel41_so1,SSL1_1p05_sel41_so2,SSL1_1p05_sel41_so3,SSL1_1p05_sel41_so4,SSL1_1p05_sel41_so5,SSL1_1p05_sel42_so0,SSL1_1p05_sel42_so1,SSL1_1p05_sel42_so2,SSL1_1p05_sel42_so3,SSL1_1p05_sel42_so4,SSL1_1p05_sel42_so5,SSL2_1p05_sel31_so0,SSL2_1p05_sel31_so1,SSL2_1p05_sel31_so2,SSL2_1p05_sel31_so3,SSL2_1p05_sel31_so4,SSL2_1p05_sel31_so5,SSL2_1p05_sel32_so0,SSL2_1p05_sel32_so1,SSL2_1p05_sel32_so2,SSL2_1p05_sel32_so3,SSL2_1p05_sel32_so4,SSL2_1p05_sel32_so5,SSL2_1p05_sel33_so0,SSL2_1p05_sel33_so1,SSL2_1p05_sel33_so2,SSL2_1p05_sel33_so3,SSL2_1p05_sel33_so4,SSL2_1p05_sel33_so5,SSL2_1p05_sel34_so0,SSL2_1p05_sel34_so1,SSL2_1p05_sel34_so2,SSL2_1p05_sel34_so3,SSL2_1p05_sel34_so4,SSL2_1p05_sel34_so5,SSL2_1p05_sel35_so0,SSL2_1p05_sel35_so1,SSL2_1p05_sel35_so2,SSL2_1p05_sel35_so3,SSL2_1p05_sel35_so4,SSL2_1p05_sel35_so5,SSL2_1p05_sel36_so0,SSL2_1p05_sel36_so1,SSL2_1p05_sel36_so2,SSL2_1p05_sel36_so3,SSL2_1p05_sel36_so4,SSL2_1p05_sel36_so5,SSL2_1p05_sel37_so0,SSL2_1p05_sel37_so1,SSL2_1p05_sel37_so2,SSL2_1p05_sel37_so3,SSL2_1p05_sel37_so4,SSL2_1p05_sel37_so5,SSL2_1p05_sel38_so0,SSL2_1p05_sel38_so1,SSL2_1p05_sel38_so2,SSL2_1p05_sel38_so3,SSL2_1p05_sel38_so4,SSL2_1p05_sel38_so5,SSL2_1p05_sel39_so0,SSL2_1p05_sel39_so1,SSL2_1p05_sel39_so2,SSL2_1p05_sel39_so3,SSL2_1p05_sel39_so4,SSL2_1p05_sel39_so5,SSL2_1p05_sel40_so0,SSL2_1p05_sel40_so1,SSL2_1p05_sel40_so2,SSL2_1p05_sel40_so3,SSL2_1p05_sel40_so4,SSL2_1p05_sel40_so5,SSL2_1p05_sel41_so0,SSL2_1p05_sel41_so1,SSL2_1p05_sel41_so2,SSL2_1p05_sel41_so3,SSL2_1p05_sel41_so4,SSL2_1p05_sel41_so5,SSL2_1p05_sel42_so0,SSL2_1p05_sel42_so1,SSL2_1p05_sel42_so2,SSL2_1p05_sel42_so3,SSL2_1p05_sel42_so4,SSL2_1p05_sel42_so5,SSL3_1p05_sel31_so0,SSL3_1p05_sel31_so1,SSL3_1p05_sel31_so2,SSL3_1p05_sel31_so3,SSL3_1p05_sel31_so4,SSL3_1p05_sel31_so5,SSL3_1p05_sel32_so0,SSL3_1p05_sel32_so1,SSL3_1p05_sel32_so2,SSL3_1p05_sel32_so3,SSL3_1p05_sel32_so4,SSL3_1p05_sel32_so5,SSL3_1p05_sel33_so0,SSL3_1p05_sel33_so1,SSL3_1p05_sel33_so2,SSL3_1p05_sel33_so3,SSL3_1p05_sel33_so4,SSL3_1p05_sel33_so5,SSL3_1p05_sel34_so0,SSL3_1p05_sel34_so1,SSL3_1p05_sel34_so2,SSL3_1p05_sel34_so3,SSL3_1p05_sel34_so4,SSL3_1p05_sel34_so5,SSL3_1p05_sel35_so0,SSL3_1p05_sel35_so1,SSL3_1p05_sel35_so2,SSL3_1p05_sel35_so3,SSL3_1p05_sel35_so4,SSL3_1p05_sel35_so5,SSL3_1p05_sel36_so0,SSL3_1p05_sel36_so1,SSL3_1p05_sel36_so2,SSL3_1p05_sel36_so3,SSL3_1p05_sel36_so4,SSL3_1p05_sel36_so5,SSL3_1p05_sel37_so0,SSL3_1p05_sel37_so1,SSL3_1p05_sel37_so2,SSL3_1p05_sel37_so3,SSL3_1p05_sel37_so4,SSL3_1p05_sel37_so5,SSL3_1p05_sel38_so0,SSL3_1p05_sel38_so1,SSL3_1p05_sel38_so2,SSL3_1p05_sel38_so3,SSL3_1p05_sel38_so4,SSL3_1p05_sel38_so5,SSL3_1p05_sel39_so0,SSL3_1p05_sel39_so1,SSL3_1p05_sel39_so

In [63]:
df.merge(pack_df, how='left', on=['lot_id','wafer_id','retx','rety']).to_csv('~/Downloads/asdf.csv')